In [1]:
from meteostream import GefsClient
import xarray as xr

In [2]:
client = GefsClient(grib_dir="/home/tylerradebaugh/data/grib")

In [5]:
client.download_gefs_prob_data()

Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f000.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f006.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f012.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f018.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f024.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f030.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f036.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f042.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f048.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f054.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f060.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t

In [3]:
client.run_gefs(remove_input=False)

Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f000.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f006.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f012.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f018.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f024.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f030.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f036.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f042.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f048.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f054.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f060.grib2
Downloaded: /home/tylerradebaugh/data/grib/gefs.wave.t

In [2]:
ds = xr.open_dataset("/home/tylerradebaugh/data/grib/selected_38_gefs.grib2", engine='cfgrib', backend_kwargs={"indexpath": None})

np.datetime64('2025-02-02T18:00:00.000000000')